# 행안부 도로명주소 API 활용

In [2]:
import requests
from urllib.parse import quote

- API key 가져오기

In [4]:
with open('roadapikey.txt') as file:
    road_key = file.read()

In [5]:
len(road_key)

40

- URL 만들기

In [7]:
# 한글은 반드시 인코딩해서 보내야 함
quote('서초구청')

'%EC%84%9C%EC%B4%88%EA%B5%AC%EC%B2%AD'

In [25]:
road_url = 'https://www.juso.go.kr/addrlink/addrLinkApiJsonp.do'
params1 = f'confmKey={road_key}&currentPage=1&countPerPage=10'
params2 = f"keyword={quote('서초구청')}&resultType=json"
url = f'{road_url}?{params1}&{params2}'

- 정상적인 경우의 코딩
<pre>
result = requests.get(url).json()
</pre>

In [33]:
result = requests.get(url)
result.status_code

200

In [34]:
import json
res = json.loads(result.text[1:-1])
res

{'results': {'common': {'errorMessage': '정상',
   'countPerPage': '10',
   'totalCount': '3',
   'errorCode': '0',
   'currentPage': '1'},
  'juso': [{'detBdNmList': '본관동,보건소,여권민원실',
    'engAddr': '2584, Nambusunhwan-ro, Seocho-gu, Seoul',
    'rn': '남부순환로',
    'emdNm': '서초동',
    'zipNo': '06750',
    'roadAddrPart2': '(서초동)',
    'emdNo': '03',
    'sggNm': '서초구',
    'jibunAddr': '서울특별시 서초구 서초동 1376-3 서초구청, 서초구의회',
    'siNm': '서울특별시',
    'roadAddrPart1': '서울특별시 서초구 남부순환로 2584',
    'bdNm': '서초구청, 서초구의회',
    'admCd': '1165010800',
    'udrtYn': '0',
    'lnbrMnnm': '1376',
    'roadAddr': '서울특별시 서초구 남부순환로 2584(서초동)',
    'lnbrSlno': '3',
    'buldMnnm': '2584',
    'bdKdcd': '0',
    'liNm': '',
    'rnMgtSn': '116502000003',
    'mtYn': '0',
    'bdMgtSn': '1165010800113760003000001',
    'buldSlno': '0'},
   {'detBdNmList': '',
    'engAddr': '46, Nambusunhwan-ro 347-gil, Seocho-gu, Seoul',
    'rn': '남부순환로347길',
    'emdNm': '서초동',
    'zipNo': '06730',
    'roadAddrPart2': '(

- res 에서 세건의 주소(roadAddr) 가져오기

In [36]:
res.keys()

dict_keys(['results'])

In [37]:
res['results'].keys()

dict_keys(['common', 'juso'])

In [39]:
len(res['results']['juso'])

3

In [40]:
res['results']['juso'][0]['roadAddr']

'서울특별시 서초구 남부순환로 2584(서초동)'

In [41]:
print(res['results']['juso'][0]['roadAddr'])
print(res['results']['juso'][1]['roadAddr'])
print(res['results']['juso'][2]['roadAddr'])

서울특별시 서초구 남부순환로 2584(서초동)
서울특별시 서초구 남부순환로347길 46(서초동)
서울특별시 서초구 양재대로12길 73-19(원지동)


- 서초구청, 강남구청, 동작구청, 용산구청, 성동구청 도로명 주소 가져오기

In [ ]:
road_url = 'https://www.juso.go.kr/addrlink/addrLinkApiJsonp.do'
params1 = f'confmKey={road_key}&currentPage=1&countPerPage=10'
params2 = f"keyword={quote('서초구청')}&resultType=json"
url = f'{road_url}?{params1}&{params2}'

In [42]:
places = '서초구청, 강남구청, 동작구청, 용산구청, 성동구청'.split(', ')
addr_list = []
for place in places:
    url = f'{road_url}?{params1}&keyword={quote(place)}&resultType=json'
    result = requests.get(url)
    res = json.loads(result.text[1:-1])         # 응급 조치
    addr = res['results']['juso'][0]['roadAddr']
    addr_list.append(addr)

In [43]:
addr_list

['서울특별시 서초구 남부순환로 2584(서초동)',
 '서울특별시 강남구 도곡로18길 7(도곡동)',
 '서울특별시 동작구 장승배기로 161(노량진동)',
 '서울특별시 용산구 백범로79길 64(청파동3가)',
 '서울특별시 성동구 고산자로 270(행당동)']

In [44]:
import pandas as pd

df = pd.DataFrame({'장소':places, '도로명주소':addr_list})
df

,장소,도로명주소
0,서초구청,서울특별시 서초구 남부순환로 2584(서초동)
1,강남구청,서울특별시 강남구 도곡로18길 7(도곡동)
2,동작구청,서울특별시 동작구 장승배기로 161(노량진동)
3,용산구청,서울특별시 용산구 백범로79길 64(청파동3가)
4,성동구청,서울특별시 성동구 고산자로 270(행당동)


In [47]:
df.to_csv('서울관공서.csv', index=False)